# ADTA Final Project Group 3: Boston Housing Dataset
#### Damaris Paez
#### Megan Palacio
#### Denia Bradley
#### Mayela Tamayo
#### Nyarai Made

### Project Description:
We have been tasked to analyze historical housing data to predict median property prices in the Boston metro area. 
Based on the given dataset, our goal is to build a linear regression model that accurately predicts the median housing prices, identifies the most influential factors, and provides investment recommendations.

### The Data:
- CRIM: per capita crime rate by town
- ZN: proportion of residential land zoned for lots over 25,000 sq.ft.
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- NOX: nitric oxides concentration (parts per 10 million)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centers
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per 10,000 dollars
- PTRATIO: pupil-teacher ratio by town
- B: 1000 x (Bk - 0.63)^2 where Bk is the proportion of Black people by town
- LSTAT: % lower status of the population
- MEDV: Median value of owner-occupied homes in $1000’s

### Data Cleaning and Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# import needed libraries

In [ ]:
df = pd.read_csv("Boston_Housing_Dataset.csv")
# load the dataset

In [ ]:
df.shape

In [ ]:
df.columns
# column names

In [ ]:
df.head()
# view the first few rows of the dataset

In [ ]:
df.tail()
# view the last few rows of the dataset

In [ ]:
df.describe().transpose()
# obtain summary statistics on the dataset

In [ ]:
df.info()
# identifying data types

#### -CHAS is a binary variable (1 or 0) since we are doing a linear regression model we will leave as an integer. If modeling requires, we can convert to categorical.

In [ ]:
df.isnull().sum()
# checking for any missing values, there are 0

In [ ]:
fig = plt.figure(figsize=(15,10))
df.boxplot()
plt.title("Boston Dataset Boxplots")
plt.tight_layout()
plt.show()
fig.savefig("Boston Dataset Boxplots.png")
# determining any outliers in the dataset
# columns like CRIM, ZN, RM, and B seem to have outliers

In [ ]:
df['B'].describe()

In [ ]:
# we will need to calculate the IQR along with the upper and lower bounds to determine the outliers

Q1 = 377.717500
Q3 = 396.157500
IQR = Q3 - Q1
print(IQR)

In [ ]:
upper = Q3 + (1.5*IQR)
lower = Q1 - (1.5*IQR)
print(upper)
print(lower)
#anything above 423.82 is an outlier but since our max value is 396.90 there are no outliers on the higher end
#anything below 350.058 will be considered an outlier

In [ ]:
outliers_b = df[(df['B'] < lower) | (df['B'] > upper)]
print(outliers_b['B'])
print(outliers_b['B'].count())
# determined the values of the outliers in column B
# There are 53 outliers in the column

In [ ]:
df.drop(outliers_b.index, inplace=True)
# Removed outliers from column B

In [ ]:
df.shape

In [ ]:
df['ZN'].describe()

In [ ]:
Q1 = df['ZN'].quantile(0.25)
Q3 = df['ZN'].quantile(0.75)
IQR = Q3 - Q1

In [ ]:
outliers_ZN = df[(df['ZN'] < (Q1 - 1.5 * IQR)) | (df['ZN'] > (Q3 + 1.5 * IQR))]
print(outliers_ZN['ZN'])
print(outliers_ZN['ZN'].count())

In [ ]:
df.drop(outliers_ZN.index, inplace=True)
# Removed outliers from column ZN

In [ ]:
df.shape

In [ ]:
# instead of removing outliers for the CRIM column, we will apply a log transformation to help reduce the impact the outliers will have on our model
df['CRIM_log'] = np.log1p(df['CRIM'])

In [ ]:
sns.histplot(df['CRIM'], kde=True)
plt.show()
#before log transformation some 
# some locations have low crime rates while others may have extremely high crime rates which creates outliers. This can impact the results of our model.

In [ ]:
#sns.histplot(df['CRIM'], kde=True)
sns.histplot(df['CRIM_log'], kde=True)
plt.show() #after log transformation
# While the distribution is still showing to be skewed after the log transformation, the data effect is less extreme. 
# With the transformation we are able to reduce the effect the outliers would have on the model while keeping the 
# data points and the relationship the variables have with one another.

In [ ]:
df.to_csv("boston_housing_cleaned_dataset.csv", index=False)

## Data Analysis and Visualization

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("boston_housing_cleaned_dataset.csv")

Below i am checking the overall distribution of housing prices and key features. this should give insights to the range, mean and stdev

In [ ]:
print(df.describe())


In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

I want to see and understand correlations. Below is a correlation matrix and heatmap to highlight stronger and weaker relationships between the variables

In [ ]:
corr_matrix = df.corr()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Feature Correlations with Housing Prices")
plt.show()

Visualisations below include "Histogram of Housing Prices", "Scatter Plot of RM vs MEDV", "Boxplot of Tax vs MEDEV"

In [ ]:
sns.scatterplot(x=df["RM"], y=df["MEDV"])
plt.xlabel("Average Number of Rooms")
plt.ylabel("Median Value of Homes")
plt.title("Impact of Rooms on Housing Prices")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(df["MEDV"], bins=30, kde=True)
plt.xlabel("Median Home Price ($1000s)")
plt.ylabel("Frequency")
plt.title("Distribution of Housing Prices in Boston")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(x=df["TAX"], y=df["MEDV"])
plt.xlabel("Property Tax Rate ($ per $10,000)")
plt.ylabel("Median Home Price ($1000s)")
plt.title("Effect of Tax Rates on Housing Prices")
plt.xticks(rotation=45)  # Rotate tax labels for better readability
plt.show()

Complaring Correlatins with MEDV below. Trying to see which factors are most correlated with it. 

In [ ]:
corr_with_medv = df.corr()["MEDV"].sort_values(ascending=False)
print(corr_with_medv)

Comparing two variables to see how they influence pricing side by side.  See all scatter plots below.

In [ ]:
sns.scatterplot(x=df["RM"], y=df["MEDV"])
plt.xlabel("Average Number of Rooms")
plt.ylabel("Median Home Price ($1000s)")
plt.title("Comparison: More Rooms vs. Higher Prices")
plt.show()

In [ ]:
sns.scatterplot(x=df["LSTAT"], y=df["MEDV"])
plt.xlabel("% Lower Status Population")
plt.ylabel("Median Home Price ($1000s)")
plt.title("Comparison: Lower Status Percentage vs. Price Decline")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.lineplot(x=df["RM"], y=df["MEDV"], label="Rooms vs. Prices", color="blue")
sns.lineplot(x=df["LSTAT"], y=df["MEDV"], label="Lower Status % vs. Prices", color="red")
plt.xlabel("Feature Value")
plt.ylabel("Median Home Price ($1000s)")
plt.title("Comparison of Influential Factors")
plt.legend()
plt.show()

## Step 3: Modeling - Linear Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
X = df.drop('MEDV', axis=1)
y = df['MEDV']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("R-squared (R²):", r2)

In [ ]:
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
coefficients

## Step 3: Modeling - Linear Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
X = df.drop('MEDV', axis=1)
y = df['MEDV']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("R-squared (R²):", r2)

In [ ]:
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
coefficients

## Step 3: Modeling - Linear Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
X = df.drop('MEDV', axis=1)
y = df['MEDV']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("R-squared (R²):", r2)

In [ ]:
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
coefficients

## Step 3: Modeling - Linear Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
X = df.drop('MEDV', axis=1)
y = df['MEDV']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("R-squared (R²):", r2)

In [ ]:
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
coefficients